### Correr EvoMSA por cada DS y con todos los SinglePreModels

- Haha21
- Exist21
- FakDes21
- MeOffendEs21
- DaVincis22
- Misogyny
- Semeval18 (anger, fear)
- MexA3t18_anger
- Detest22
- Detoxis21

In [2]:
from  threading import Thread
from time import sleep, perf_counter

In [3]:
from EvoMSA.base import EvoMSA
from microtc.utils import load_model
from glob import glob
import pandas as pd
from sklearn import metrics

In [28]:
from EvoMSA import __version__ as version
print(version)

1.3.6


In [7]:
stacked_method = "sklearn.svm.LinearSVC"
# usar = "sklearn.naive_bayes.BernoulliNB"
# usar = "sklearn.naive_bayes.MultinomialNB"

davincis22 = [load_model('../../single_models/davincis22_single_Es.evomsa'), stacked_method]
exist22 = [load_model('../../single_models/exist21_single_Es.evomsa'), stacked_method]
fakedes2021 = [load_model('../../single_models/fakedes2021_single_Es.evomsa'), stacked_method]
haha21 = [load_model('../../single_models/haha21_single_Es.evomsa'), stacked_method]
meoffendes21 = [load_model('../../single_models/meoffendes21_single_Es.evomsa'), stacked_method]
mexa3t2018_aggress = [load_model('../../single_models/mexa3t2018_aggress_single_Es.evomsa'), stacked_method]
misogyny = [load_model('../../single_models/misogyny_single_Es.evomsa'), stacked_method]
semeval2018_anger = [load_model('../../single_models/semeval2018_anger_single_Es.evomsa'), stacked_method]
semeval2018_fear = [load_model('../../single_models/semeval2018_fear_single_Es.evomsa'), stacked_method]

mexa3t_model = [load_model('../../models/mexa3t2018_aggress_Es.evomsa'), stacked_method]
misoginia_model = [load_model('../../models/misoginia_Es.evomsa'), stacked_method]

mm = [davincis22, exist22, fakedes2021, haha21, meoffendes21, mexa3t2018_aggress, misogyny, semeval2018_anger, semeval2018_fear]
#evo = EvoMSA(TR=True, B4MSA=False, lang='es', Emo=True, Aggress=True, stacked_method=stacked_method, models= mm)

#for m in mm:
#    print('> ', m)
#    evo = EvoMSA(TR=True, B4MSA=False, lang='es', Emo=True, Aggress=True, stacked_method=stacked_method, models= m)

C:\Users\juan\anaconda3\envs\orange3\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator LinearSVC from version 0.22.1 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\Users\juan\anaconda3\envs\orange3\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator GaussianNB from version 0.22.1 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [8]:
def task(fn):
    
    train_df = pd.read_json(fn, lines=True)
    test_df = pd.read_json(fn.replace("_train", "_test"), lines=True)

    X_train = train_df[['text']]
    y_train = train_df[['klass']]
    
    X_test = test_df[['text']]
    y_test = test_df[['klass']]
    
    ds_n = fn.replace("../../dataset_Es\\",'').replace('_Es_train.json','')
    
    print(ds_n)
    
    # single
    evo = EvoMSA(TR=True, B4MSA=False, lang='es', Emo=True, Aggress=True, stacked_method=stacked_method, models= [])
    evo.fit(X_train.squeeze(), y_train.squeeze())
    pred = evo.predict(X_test.squeeze())
    recall_score_single = metrics.recall_score(pred, y_test, average="macro")
    f1_score_single = metrics.f1_score(pred, y_test, average="macro")
    
    # multiple
    evo = EvoMSA(TR=True, B4MSA=False, lang='es', Emo=True, Aggress=True, stacked_method=stacked_method, models= mm)
    evo.fit(X_train.squeeze(), y_train.squeeze())
    pred = evo.predict(X_test.squeeze())
    recall_score_multiple = metrics.recall_score(pred, y_test, average="macro")
    f1_score_multiple = metrics.f1_score(pred, y_test, average="macro")

    _ = [ds_n, X_train.shape, recall_score_single, f1_score_single, recall_score_multiple, f1_score_multiple]
    
    results.append(_)


In [ ]:
fnames = glob("../../dataset_Es/*_train.json")
fnames.sort()

results = []
start_time = perf_counter()
threads = []
for fn in fnames:
    
    tsk = Thread(target=task, args=(fn,))
    threads.append(tsk)
    tsk.start()

for tsk in threads:
    tsk.join()

end_time = perf_counter()

print(f'It took {end_time- start_time: 0.2f} second(s) to complete.')

In [ ]:
print(results)

In [12]:
#from microtc.utils import save_model
#save_model(results, 'resullts_multiple_01.metrics')

In [27]:
lst = load_model('../results/resullts_multiple_01.metrics')
df = pd.DataFrame(lst, columns=['DataSet', 'shape', 's-recall', 's-f1', 'm-recall', 'm-f1'])
df['diff'] = df['s-f1'] - df['m-f1']
df

,DataSet,shape,s-recall,s-f1,m-recall,m-f1,diff
0,semeval2018_sadness,"(1350, 1)",0.360507,0.408195,0.424273,0.430897,-0.022702
1,semeval2018_joy,"(1260, 1)",0.512070,0.481660,0.502229,0.479207,0.002452
2,semeval2018_anger,"(1359, 1)",0.420269,0.396493,0.422241,0.419181,-0.022687
3,semeval2018_fear,"(1368, 1)",0.528177,0.482220,0.490019,0.473300,0.008920
4,semeval2018_valence,"(1795, 1)",0.334615,0.322807,0.347622,0.334407,-0.011600
5,meoffendes2021,"(3795, 1)",0.793131,0.755097,0.738193,0.675904,0.079193
6,exist2021,"(2655, 1)",0.746018,0.745777,0.741563,0.740359,0.005419
7,davincis2022,"(2521, 1)",0.770896,0.769870,0.771220,0.767608,0.002262
8,mexa3t2018_aggress,"(5389, 1)",0.793277,0.789327,0.801422,0.794541,-0.005214
9,haha2021,"(18000, 1)",0.842803,0.836869,0.852546,0.837146,-0.000277


- Cada renglon es un Dataset (train.json)
- single: sin usar pre-modelos EvoMSA
- multiple: usando todos los pre-modelos EvoMSA (No se aplicoo: excepto el evaluado)